# Tutorial on how to fit diameters with PMOIRED: uniform, limb-darkened, oblate, spotted

We will analyse the CHARA/MIRC data of CL Lac / IRC+50448, a Mira type AGB star. The data, which have been published in [Chiavassa et al. 2020](https://ui.adsabs.harvard.edu/abs/2020A%26A...640A..23C/abstract), are available on [OIdB](https://oidb.jmmc.fr/search.html?conesearch=IRC%2B50448%2CJ2000%2C2%2Carcmin&perpage=50&instrument=MIRC&cs_radius_unit=arcmin&cs_equinox=J2000&order=t_min&caliblevel=3&category=SCIENCE&cs_radius=2&cs_position=IRC%2B50448).

In this tutorial, you should:
- load the data and display them
- fit a uniform disk model
- fit a limb-darkenend disk model
- add oblatness to the star shape
- add a spot on the star surface

PMOIRED: https://github.com/amerand/PMOIRED - tutorial by amerand@eso.org, Feb 2023


In [1]:
try:
    %matplotlib pylab
    print('pylab')
except:
    %matplotlib widget
    print('widget')
    
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import os
try:
    # -- global installation",
    import pmoired
    from pmoired import tellcorr
    print('global installation')
except:
    # -- local installation
    import sys
    # -- where is your local version of PMOIRED
    sys.path = ['../pmoired'] + sys.path
    import __init__ as pmoired
    import tellcorr
    import dpfit
    print('local installation')
import CL_Lac

def showAnswer(x):
    print('### solution ###')
    print(x)
    print('### execution ###')
    exec(x, globals())

widget
[P]arametric [M]odeling of [O]ptical [I]nte[r]ferom[e]tric [D]ata https://github.com/amerand/PMOIRED
local installation


# Load and preview files

OIFITS data are in `./CL_Lac`. use `oi = pmoired.OI(...)` to load the files and construct your object `oi`. the constructor takes simply a list of file names. `oi.show()` will show the all the data. interesting option: `logV=True` to show visibilities (amplitude or squared) in log scale. `showFlagged=True` to show flagged data (i.e. not taken into account).

to see / execute answer: ```showAnswer(CL_Lac.load)```

In [2]:
#showAnswer(CL_Lac.load)

# Uniform disk fit

Using `oi.setupFit`, define the context of your fit: fit the `V2` data, and you can use `'max relative error':{'V2':0.5}` to ignore the squared visibilities with uncertainites larger than 50%. Then use `oi.doFit` to fit the data. To see how write models as dictionnaries in `PMOIRED`, please refer to [the model definition notebook](https://github.com/amerand/PMOIRED/blob/master/examples/Model%20definitions%20and%20examples.ipynb).


to see / execute answer: ```showAnswer(CL_Lac.udfit)```

In [3]:
#showAnswer(CL_Lac.udfit)

# Limb darkened disk using 4-parameters

from [Claret (2000)](https://ui.adsabs.harvard.edu/abs/2000A%26A...363.1081C), table [J/A+A/363/1081/atlas](https://vizier.cds.unistra.fr/viz-bin/VizieR-3?-source=J/A%2bA/363/1081/atlas&-out.max=50&-out.form=HTML%20Table&-out.add=_r&-out.add=_RAJ,_DEJ&-sort=_r&-oc.form=sexa) get the 4-coef CLD parameters: $I(\mu)/I(1) = 1-\sum_{k=1}^{4}a_k(1-\mu^{k/2})$. In this context, $\mu = \cos(\gamma) = \sqrt{1-r^2}$, $\gamma$ being the angle between the line of sight and the emergent intensity and $r$ the normalised radial distance from the centre of the star to its limb. 

Based on the stellar parameters in [Chiavassa et al. 2020](https://ui.adsabs.harvard.edu/abs/2020A%26A...640A..23C/abstract), you can use Teff=3500K and logg=1.0 in the H band for MIRCX.

reminder: in `PMOIRED`, you can describe a disk with arbitrary profile using `diam` and `profile`. The diameter is in milliarcseconds, and the profile is a string using special names `$R` and `$MU` and any additional parameters you need: a linear limb-darkened disk, parametrised with `u`, will be entered in` PMOIRED` as `{'diam':2.0, 'profile':'1-$u*(1-$MU)', 'u':0.1}`.

You should first fix the LD parameters (using `oi.doFit`'s option `doNotFit=[...]` to list the parameters you do not want to fit), then try to fit them: the fot does not converge. You can inspect the fit with `oi.showfit()` which shows the evolution of the parameters during the fitting: use the mouse too zoom and inspect the convergence.

to see / execute answer: ```showAnswer(CL_Lac.ldc2000fit)```

In [4]:
#showAnswer(CL_Lac.ldc2000fit)

## Adding prior to help fit the LD parameters
To help fit the LD parameters, we can add the constrain that $|a_k|<2$ for instance. This is done using the `prior` keyword in `doFit`: we pass a list of priors as tuples: `prior=[('diam', '<', 3), ...]` (no $ where you refer to parameters!). 

Question: Can you tell if the fit is reliable? why?

to see / execute answer: ```showAnswer(CL_Lac.ldc2000fitprior)```

In [5]:
#showAnswer(CL_Lac.ldc2000fitprior)

# Limb darkening: power law

To fit the limb darkening, we need a simpler law (with less parameters): we can use a power law as described in [Hestroffer (1997)](https://ui.adsabs.harvard.edu/abs/1997A%26A...327..199H/abstract): $I[\mu]/I(1) = \mu^\alpha$. To be able to fit the LD coefficient (`alpha`), one needs data in the second lobe of visibility, which is our case...

to see / execute answer: ```showAnswer(CL_Lac.ldalphafit)```

In [6]:
#showAnswer(CL_Lac.ldalphafit)

## Limb darkened oblate star

We can make the stellar shape oblate using `incl` and `projang`: `incl` is the "inclination", which means that the stellar shape will be an ellipse with large axis will have diameter `diam` and small axis cos(`incl`)$\times$`diam`. The large axis orientation is set by `projang`: 0 for North and 90 for East. All angles are in degrees.

to see / execute answer: ```showAnswer(CL_Lac.ldalphaoblatefit)```

In [7]:
#showAnswer(CL_Lac.ldalphaoblatefit)

# Adding a spot to the limb-darkened oblate model

Add a spot to the previous best fit model (as a uniform disk for example). The spot must be able to be at different position on the star: use `x` and `y` (in mas). Also you should give it a flux `f` (total flux, not surface brightness!). 

Fit the `T3PHI` and `V2` data. if the spot gets too small, you can use a prior to force its size to be a reasonable fraction of the stellar size (between ~1/2 and ~1/4 of the size of the star, considering we have data in the third lobe).

You may find that the fit does not converge: it is because it is sensitive to the initial conditions, in particular the position of the spot. Try different initial positions.

to see / execute answer: ```showAnswer(CL_Lac.ldspotfit)```

In [8]:
#showAnswer(CL_Lac.ldspotfit)

## `gridFit` to find the global best position for the spot

We saw the fit is sensitive to the initial position of the spot. Use `oi.gridFit()` to explore various initial positions. You can use `constrain=` to give a list of constrain on the initial parameters, and `prior=[(...)]` which will be used while optimising the model.

compare you result to the ones presented in [Fig 2](https://www.aanda.org/articles/aa/full_html/2020/08/aa37832-20/F2.html) of the publication. 

to see / execute answer: ```showAnswer(CL_Lac.ldspotgrid)```

In [9]:
#showAnswer(CL_Lac.ldspotgrid)

# Use bootstrapping to evaluate the uncertainties

Use `oi.bootstrapFit(Nfits)` to perform `Nfits` fit with resampled data. `Nfits` should be of the order of the (number of baselines + number of triangle)x(number of files). In our case, the is 540. Although the bootstrapping is parallelised, running 540 fits will take several minutes on a typical laptop. You can use a smaller number, e.g. 100, to get an idea of the result. `oi.showBootstrap()` let you see the result as a corner plot.

to see / execute answer: ```showAnswer(CL_Lac.bootstrap)```

In [10]:
#showAnswer(CL_Lac.bootstrap)